# Кейс 1. Прогнозирование конверсии рекламного показа в установку мобильного приложения для рекламного брокера

**Компания:** рекламный брокер, который закупает показы рекламы в мобильных приложениях и перепродает их рекламодателям.

**Продукт:** брокерская рекламная платформа, которая должна помогать рекламодателям точнее таргетировать аудиторию и снижать издержки на неэффективные показы. Решение задачи прогнозирования конверсии встроено в рекомендательный движок брокера и позволяет в реальном времени определять, стоит ли показывать конкретному пользователю ту или иную рекламу.

**Проблема продукта.** Показы рекламы стоят денег, но не каждый показ приводит к установке приложения. В результате рекламодатель тратит средства на неэффективный трафик. 


## Условия и информация для решения
**Вам предоставляется датасет:**

[Скачать датасет](https://lms.mipt.ru/pluginfile.php/272164/mod_page/content/3/advertisers.csv)

В таблице содержится история показов рекламы в мобильных приложениях. 

Столбцы:

install — конверсия показа в установку;

user id — id пользователя, которому была показана реклама;

game app — id приложения, в котором была показана реклама;

country — страна пользователя;

advertiser — id рекламного аукциона, который выбирает рекламу для показа;

age — возрастная группа пользователя (1: 18–25, 2: 26–35, 3: 36–45, 4: 46–55, 5: 55+)

user quality score — уровень «качества» пользователя, выставленный отдельной ML-моделью (1 — низкая вероятность конверсии, 2 — средняя, 3 — высокая);

сreatedat — дата показа рекламы;

spending — стоимость показа рекламы;

earning — плата от рекламодателя за установку;

mccmnc — оператор мобильной связи пользователя.

**В рамках решения кейса вам будет необходимо:**

Изучить датасет, провести EDA, предоставить бизнес-заказчикам практические инсайты, поддерживающие принятие решений на основе данных.

Сформулировать аналитические выводы для ML-отдела для последующей разработки модели, прогнозирующей прибыльность рекламного показа.

Подготовить дашборд, который содержит графики и визуализации, отражающие ключевые выводы анализа данных. Он должен демонстрировать выявленные закономерности и инсайты, быть понятным для пользователей без технической подготовки и служить инструментом для принятия решений на основе данных.

Подготовить все артефакты работы и презентацию с решения:

Ноутбук .ipynb с расчетами и визуализациями инсайтов.

Дашборд на платформе gradio.app.

Презентацию для защиты решения (.pdf).

Вопросы к консультации:
1. В условии задания нет описания столбца carrier.
2. 

# Импорты библиотек

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
# Добавьте необходимые библиотеки здесь

# Чтение данных

In [3]:
df_path = "advertisers.csv"

df = pd.read_csv(df_path)

df.head()

,install,user id,game app,country,advertiser,age,user quality score,createdat,spending,earning,carrier,mccmnc
0,0,user_045828,game_2,AU,advertiser_01,1,1,1675578699,0.014110,0.0,carrier_11,505244
1,0,user_150401,game_4,GB,advertiser_22,5,2,1676420955,0.005462,0.0,carrier_1,234014
2,0,user_102674,game_4,DE,advertiser_07,1,3,1676034552,0.020445,0.0,carrier_17,262203
3,0,user_290419,game_4,AU,advertiser_11,5,3,1677552033,0.010014,0.0,carrier_0,505604
4,0,user_070536,game_5,DE,advertiser_13,4,2,1675775984,0.014186,0.0,carrier_7,262897


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   install             50000 non-null  int64  
 1   user id             50000 non-null  object 
 2   game app            50000 non-null  object 
 3   country             50000 non-null  object 
 4   advertiser          50000 non-null  object 
 5   age                 50000 non-null  int64  
 6   user quality score  50000 non-null  int64  
 7   createdat           50000 non-null  int64  
 8   spending            50000 non-null  float64
 9   earning             50000 non-null  float64
 10  carrier             50000 non-null  object 
 11  mccmnc              50000 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 4.6+ MB


In [14]:
df['createdat_dt'] = pd.to_datetime(df['createdat'], unit='ms')
df.describe()

,install,age,user quality score,createdat,spending,earning,mccmnc,createdat_dt
count,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000
mean,0.022040,2.960500,1.983440,1.676420e+09,0.013184,0.013619,382281.645140,1970-01-20 09:40:19.586094619
min,0.000000,1.000000,1.000000,1.675210e+09,0.000220,0.000000,232014.000000,1970-01-20 09:20:09.637000
25%,0.000000,2.000000,1.000000,1.675816e+09,0.001081,0.000000,234783.000000,1970-01-20 09:30:16.011750
50%,0.000000,3.000000,2.000000,1.676415e+09,0.012745,0.000000,312573.000000,1970-01-20 09:40:14.869000
75%,0.000000,4.000000,3.000000,1.677023e+09,0.020103,0.000000,505231.000000,1970-01-20 09:50:22.866250
max,1.000000,5.000000,3.000000,1.677629e+09,0.044000,2.000000,724983.000000,1970-01-20 10:00:28.697000
std,0.146815,1.425251,0.815806,6.977229e+05,0.010679,0.117121,167495.115808,NaN


In [20]:
df.describe(include=['object'])

,user id,game app,country,advertiser,carrier
count,50000,50000,50000,50000,50000
unique,50000,7,7,30,20
top,user_045828,game_0,AT,advertiser_21,carrier_8
freq,1,8375,7272,1760,2582


In [25]:
for column in df:
    print(column, df[column].nunique(), sep=': ')

install: 2
user id: 50000
game app: 7
country: 7
advertiser: 30
age: 5
user quality score: 3
createdat: 49479
spending: 204
earning: 204
carrier: 20
mccmnc: 413
createdat_dt: 49479


In [26]:
for column in df:
    if df[column].nunique() < 10:
        print(column, df[column].unique(), sep=': ')

install: [0 1]
game app: ['game_2' 'game_4' 'game_5' 'game_0' 'game_3' 'game_1' 'game_6']
country: ['AU' 'GB' 'DE' 'AT' 'BR' 'IN' 'US']
age: [1 5 4 3 2]
user quality score: [1 2 3]


In [19]:
df['createdat_dt'].dt.strftime('%Y-%m-%d %H').unique()

array(['1970-01-20 09', '1970-01-20 10'], dtype=object)

# Чистка данных

In [ ]:
# Ваш код здесь

# Гипотезы

In [ ]:
# Укажите список + стратегию проверки